## FISH Image Converter-local (version 1.0)

In [1]:
# Import packages
import os
import shutil
from smb.SMBConnection import SMBConnection
import tempfile
import re

import matplotlib.pyplot as plt
import numpy as np

# import tifffile
import bigfish
import bigfish.stack as stack
import bigfish.plot as plot
import bigfish.multistack as multistack
import bigfish.detection as detection

from fpdf import FPDF
import os
import glob
from datetime import datetime


#### Input Image info:

In [2]:
your_name = '' #your_name

#specify the location of your data on the NAS
input_directory = ' ' #/your/input/directory
output_directory = ' ' #/your/output/directory

#specify what you have in every channel
Cy5 = " " # your 670 channel (asymmetric control)
mCherry = " " # your 610 channel (query RNA)
FITC = " " # your protein channel (protein markers)
DAPI = " " 

# NAS credentials and connection information
nas_host = ' ' #your_nas_server_hostname_or_ip'
nas_user = ' ' #your_username
nas_password = ' ' #your_password
nas_share = ' ' #(specific for the Nishimura lab - edit accordingly)

#### Connect to the NAS

In [3]:
experiment_ID = input_directory.split(os.path.sep)[-1]
output_directory = os.path.join(output_directory, experiment_ID, 'images')


# Connect to the NAS
conn = SMBConnection(nas_user, nas_password, 'python-client', nas_host, use_ntlm_v2=True)
conn.connect(nas_host, 445)

# Check if connection is successfully established
smb_directory_path = os.path.relpath(input_directory, '/Volumes/WormHole')
shared_files = conn.listPath(nas_share, smb_directory_path) # move into folder with raw data
if shared_files:
    print("Connection is established.")
else:
    print("Connection is not established.")
    
# #print the filenames, save in a list 
# filenames = [shared_file.filename for shared_file in shared_files if shared_file.filename.endswith(("_R3D.dv","_R3D_REF.dv"))]
# filenames = sorted(filenames)
# filenames

Connection is established.


## <font color='red'>STOP!</font> Do not edit below this line

### 1. Code to read images directly from the NAS and save as a png

#### 1.1 List file paths and read image stacks

In [4]:
#List all the color images and brightfield images in separate lists
image_color_paths = []
bf_paths = []

for shared_file in shared_files:
    file_path = os.path.join(input_directory, shared_file.filename) 
    if shared_file.filename.endswith("_R3D_D3D.dv"):
        # If it does, append it to the image_colors_paths list
        image_color_paths.append(file_path)
        image_color_paths = sorted(image_color_paths)
    elif shared_file.filename.endswith("_R3D_REF.dv"):
        # If it ends with "_R3D_REF.dv", append it to the bf_paths list
        bf_paths.append(file_path)
        bf_paths = sorted(bf_paths)

#Read all the image_color stacks and extract the image identifiers to save in "subdirectories"
all_image_color_stacks = []
subdirectories = []
for image_color_path in image_color_paths:
    image_color_stack = stack.read_dv(image_color_path, sanity_check=False)
    all_image_color_stacks.append(image_color_stack)
    file_name = os.path.splitext(os.path.basename(image_color_path))[0]
    parts = file_name.split('_')
    numeric_part = None
    for i, part in enumerate(parts):
        if part == 'R3D':
            numeric_part = parts[i - 1]
            break
    if numeric_part is not None:
        subdirectories.append(numeric_part)

#Read all the bf stacks
all_bf_stacks = []
for bf_path in bf_paths:
    bf_stack = stack.read_dv(bf_path, sanity_check=False)
    all_bf_stacks.append(bf_stack)

#### 1.2 Split channels, max project and save in image subdirectory

In [5]:
# Now iterate through all_image_color_stacks and extract every channel, then store it in all_color_channel_list 
all_color_channel_list = []
for i, stack in enumerate(all_image_color_stacks):
    image_colors = all_image_color_stacks[i]
    
    # Assuming image_colors is a 4D array where the first dimension represents different channels
    for channel_index in range(image_colors.shape[0]):
        current_image = image_colors[channel_index, :, :]
        all_color_channel_list.append(current_image)
        

        # Create a subdirectory to save each image as a PNG file
        output_subdirectory = os.path.join(output_directory, f"{subdirectories[i]}")
        os.makedirs(output_subdirectory, exist_ok=True)

        channel_name = None
        if channel_index == 0:
            channel_name = Cy5
        elif channel_index == 1:
            channel_name = mCherry
        elif channel_index == 2:
            channel_name = FITC
        elif channel_index == 3:
            channel_name = DAPI
 
        # Save each image as a PNG file
        plot_filename = os.path.join(output_subdirectory, f"{subdirectories[i]}_{channel_name}_deconvolved.png")
        plt.imsave(plot_filename, np.max(current_image, axis=0), cmap='gray')  # Adjust cmap if needed


# Print the directory where the images are stored
print(f"{len(subdirectories)} images saved as png in {output_directory}")

0 images saved as png in /Volumes/home/01_imaging/231004_WNT002_erm-1_set-2/images


#### 1.3 Generate pdf report with deconvolved images (as .png)

In [6]:
import os
import glob
from fpdf import FPDF
from datetime import datetime

# Function to generate the cover letter
def add_cover_letter(experiment_details, readme_content):
    pdf.add_page()
    pdf.set_font("Arial", size=14)
    pdf.cell(100, 10, txt=(f"Experiment Title: {experiment_ID}"), ln=True, align='C')
    pdf.ln(2)

    # Add experiment details
    for detail in experiment_details:
        pdf.cell(200, 10, txt=detail, ln=True, align='L')
    
    pdf.ln(5)

    # Add README content
    pdf.multi_cell(0, 10, txt=readme_content)
    pdf.ln(10)

# Create a PDF report
pdf_filename = os.path.join(output_directory, "report.pdf")
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# Find the file that ends with "README"
readme_file = None
for file_path in glob.glob(os.path.join(input_directory, '*README.*')):
    readme_file = file_path
    break

if readme_file:
    # Read README content
    with open(readme_file, "r") as file:
        readme_content = file.read()
else:
    readme_content = "No README file found."

# Add cover letter once at the beginning
today_date = datetime.now().strftime("%B %d, %Y")
add_cover_letter([f"Date of processing: {today_date}",
                  f"Processed by: {your_name}",
                  " ",
                 "Readme file:"], readme_content)

# Add a single page for all images
pdf.add_page()
pdf.set_font("Arial", size=12)

# Iterate through subdirectories and add images
for subdirectory in sorted(subdirectories):
    # Add experiment title and image ID
    pdf.cell(200, 10, txt=f"Image ID: {experiment_ID}_{subdirectory}", ln=True, align='L')
    pdf.ln(5)

    # Get PNG files in the subdirectory
    subdirectory_path = os.path.join(output_directory, subdirectory)
    png_files = glob.glob(os.path.join(subdirectory_path, '*.png'))

    # Calculate the number of columns that can fit on the page
    max_columns = 4  # You can adjust this value based on your preference
    num_columns = min(len(png_files), max_columns)

    # Calculate the width of each column
    column_width = 190 / num_columns

    # Add images to the PDF in columns with space between images
    for i, png_file in enumerate(sorted(png_files)):
        x_position = pdf.get_x() + i % num_columns * column_width
        y_position = pdf.get_y() + int(i / num_columns) * 20  # Adjust space between rows

        pdf.image(png_file, x=x_position, y=y_position, w=column_width)
    
    pdf.ln(80 * ((len(png_files) - 1) // num_columns + 1))  # Adjust line height based on the number of rows

# Output the PDF file
pdf.output(pdf_filename)
print(f"PDF report created: {pdf_filename}")


FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/home/01_imaging/231004_WNT002_erm-1_set-2/images/report.pdf'

### Code by Naly Torres. Last edited March 14th. 2024